This notebook is belong to [here]( https://www.kaggle.com/competitions/amex-default-prediction/discussion/329436).

データ重すぎてメモリに乗らない。圧縮方法まとめは[ここ](https://www.kaggle.com/competitions/amex-default-prediction/discussion/328054)。+[補足](https://www.kaggle.com/competitions/amex-default-prediction/discussion/330347)  

# 自作データ読み込み 
## 背景(課題)
1. メモリ
→ 軽くすると計算速度・メモリ消費量が改善される。
2. 読み込み
→ 実行が早くなる。

## 実装
pickleを読み込んでdtypeをfloat64->float32変更してdfを返す。

## 課題
* pickleがあることを前提として読み込み部分を実装している。→csv_to_pickleは必要?  
* dtype変更はfloat64の特徴量をeda.pyにべた書きしている。→ 型を勝手に読み込んで特徴をピックアップする？これをやると64→32をやってもいいのかという判定が必要で面倒そう.
* 分割されたpickleファイルをまとめて読み込むようにする。以下イメージ

```

df = eda.read_pickles(dirname)

dirname = 'pickle/'

pickle  --- train_1.pickle
        | - train_2.pickle
        | - train_3.pickle
        | - train_4.pickle
        .
        .
        .

```


In [6]:
import eda
import pandas as pd
#df = pd.read_csv('dataset/train_data.csv')
pickle = 'pickle/train_data_0.pkl'
df1 = eda.read_pickle(filename=pickle)
print(df1.info())
df2 = pd.read_pickle(pickle)
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 190 entries, customer_ID to D_145
dtypes: float32(185), int64(1), object(4)
memory usage: 743.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 190 entries, customer_ID to D_145
dtypes: float64(185), int64(1), object(4)
memory usage: 1.4+ GB
None


## Simple Feature Engineering ([参考](https://www.kaggle.com/competitions/ieee-fraud-detection/discussion/108575))
### Train and Test
訓練とテストを分離する。今回は不要。
### NAN processing
nanの処理。参考文献では例として-999を代入している。LGBM(=Light GBM?)だとアルゴリズムがうまく捌いてくれるらしい。
### Label Encode/ Factorize/ Memory reduction
全体を通して文字列特徴量の扱い方を説明している。
* pandasのfactorizeを使うと文字列をa→1, b→2, c→3...のように変換してくれる。([例](https://qiita.com/QUANON/items/08a65012366abd150172))
* データタイプを落とすとデータの圧縮になる(float64→float32, int64→int32,...)  

### Categorical Features
LGBMはカテゴリ変数を学習可能なので、df.astype('category')のように型を指定してもよい。(ラベルエンコードしない場合)

### Splitting
主に文字列特徴量で、"Mac OS X 10_9_5"のような特徴量を"Mac OS X"(OS名)と"10_9_5"(バージョン)に分離。

### Combining / Transforming / Interaction
* 文字列特徴量であれば連結できる. 
```
df[new_column] = df[str_column] + df[str_column2]
```
* 数値特徴量であれば計算して利用できる.
```
df[new_column] = df[int_column] * df[int_column2]
```

### Frequency Encoding
カテゴリ変数をデータ全体での頻出回数でエンコードする方法。

### Aggregations / Group Statistics
```
temp = df.groupby('card1')['TransactionAmt'].agg(['mean'])   
 .rename({'mean':'TransactionAmt_card1_mean'},axis=1)
df = pd.merge(df,temp,on='card1',how='left')
```
カテゴリ変数でグループ化して数値特徴量を加工(変換)する。  
上の式では、'card1'でグループ化して'TransactionAmt'の平均値を新規特徴量として生成している。
```
   card1 TransactionAmt TransactionAmt_card1_mean
 0 a     2              2.5
 1 b     4              4
 2 a     3              2.5
```
参考: [groupby](https://qiita.com/propella/items/a9a32b878c77222630ae), [agg](https://note.nkmk.me/python-pandas-agg-aggregate/)

### Normalize / Standardize
標準化・正規化(省略)

### Outlier Removal / Relax / Smooth / PCA
異常値の削除。この記事のコンテストでは異常値を発見したかったそうで、平滑化(Relax?)を使用。頻出度が全体の0.1%未満の特徴量(カテゴリ変数の話っぽい)を-9999に変換するなど.

## More about Aggregations / Group Statistics
上のAggregations重要って言ってるだけっぽい。

## Bruteforce Feature Engineering
足して引いてかけて割って相関高いやつを消す。

## Stacking

In [2]:
# モデルたくさん作って予測値を線形回帰とかでくっつける。特徴量関係ないじゃん

## Pseudolabeling

In [ ]:
# モデル作ってテストデータを予測。予測した値を疑似ラベルとしてトレインデータにする。特徴量関係ないじゃん